In [1]:
from datetime import datetime
import itertools
import json
import math
import os
import time
import string

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import shapely.wkt
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
try:
    from urllib.request import Request, urlopen  # Python 3
except:
    from urllib2 import Request, urlopen  # Python 2
TEST_ORIGIN = '530330001003'
CLASS_LIST = ["urban village", "citywide", "destination park", "supermarket", "library", 
              "hospital", "pharmacy", "post_office", "school", "cafe"]

In [2]:
def proximity_ratio(df_destinations):
    
    # proximity ratio = no. trips under 2 miles / no. trips between 2 and 10 miles; rows with zero denominators are removed
    df_destinations['dist_under_2'] = np.where(df_destinations['distance'] < 2.0, 1, 0)
    df_destinations['dist_2_to_10'] = np.where((df_destinations['distance'] >= 2.0) & (df_destinations['distance'] < 10.0), 1, 0)
    df_blockgroup = df_destinations.groupby(['origin'], as_index=False).agg({'dist_under_2':sum,'dist_2_to_10':sum})
    
    df_blockgroup = df_blockgroup[df_blockgroup['dist_2_to_10'] != 0]   
    df_blockgroup['proximity_ratio_test'] = df_blockgroup['dist_under_2'] / df_blockgroup['dist_2_to_10']
    
    return(df_blockgroup[['origin', 'proximity_ratio_test']])

In [3]:
def vert_hori_ratio(df_destinations, df_blockgroup):
    
    df_destinations['vertical_horizontal_ratio_test'] = pd.DataFrame(np.abs( (df_destinations['dest_lat'] - df_destinations['orig_lat']) / (df_destinations['dest_lng'] - df_destinations['orig_lng']) ))
    df_blockgroup2 = df_destinations.groupby(['origin'], as_index=False)['vertical_horizontal_ratio_test'].mean()
    result_merged = pd.merge(left=df_blockgroup, right=df_blockgroup2, how='inner', left_on='origin', right_on='origin')
    
    return(result_merged)

In [4]:
def average_distance(df_destinations, df_blockgroup):
    
    df_blockgroup2 = df_destinations.groupby(['origin'], as_index=False)['distance'].mean()
    result_merged = pd.merge(left=df_blockgroup, right=df_blockgroup2, how='inner', left_on='origin', right_on='origin')
    result_merged.rename(columns = {'distance': 'average_distance_test'}, inplace=True)
    
    return(result_merged)

In [5]:
"""
Next (Optional in case above three are not good enough) - Moran's I for x and y axes. 
Create weights by pairwise distance matrix for each blockgroup (inverse probability weighting by distance), 
and calculate Moran's I using python package
"""

"\nNext (Optional in case above three are not good enough) - Moran's I for x and y axes. \nCreate weights by pairwise distance matrix for each blockgroup (inverse probability weighting by distance), \nand calculate Moran's I using python package\n"

In [6]:
"""
If we have more time, we can try to compute spatial covariance matrix (might need to normalize the matrix)
and compare a. largest eigenvalues, b. traces, c. determinants.
However, there might be some issues with interpretation (meaning of the measures). 
Also we might need to parallelize the process as it is computationally expensive. 
"""

'\nIf we have more time, we can try to compute spatial covariance matrix (might need to normalize the matrix)\nand compare a. largest eigenvalues, b. traces, c. determinants.\nHowever, there might be some issues with interpretation (meaning of the measures). \nAlso we might need to parallelize the process as it is computationally expensive. \n'

In [7]:
def prepare_psrc(psrc_raw):
    
    # this will compute three features using PSRC data; just need to run it once
    psrc_blockgroup = proximity_ratio(psrc_raw)
    with_vert_hori_ratio = vert_hori_ratio(psrc_raw, psrc_blockgroup)
    with_average_distance = average_distance(psrc_raw, with_vert_hori_ratio)
    
    return (with_average_distance)    

In [8]:
def evaluate_features(google_input, psrc_input, basket_combination):
    
    """
    This is our main function to compare google API with PSRC data, 
    by comparing three features. 
    
    Input: 
        1. google API data (each row represents one trip)
        2. processed PSRC data (each row represents a blockgroup)
        3. basket combination
        
    The output is a list of three MSEs. 
    """
    
    
    # Filter to match basket parameters based on rank (distance from destination)
    for i in range(len(CLASS_LIST)):
        filtered_data = google_input[(google_input['class'] != CLASS_LIST[i]) | (google_input['rank'] <= basket_combination[i])]
    
    # FEATURES: PROXIMITY RATIO, VERTICAL/HORIZONTAL TRAVEL DISTANCES, AVERAGE DISTANCE TO DESTINATION
    
    # Creating google results
    with_proximity_ratio = proximity_ratio(filtered_data.copy())
    with_vert_hori_ratio = vert_hori_ratio(filtered_data.copy(), with_proximity_ratio)
    with_average_distance = average_distance(filtered_data, with_vert_hori_ratio)


    # merge with proximity scores from PSRC
    df_googleAPI = with_average_distance
    #df_psrc = prepare_psrc(psrc_input)
    df_psrc_reduced = psrc_input.loc[psrc_input['origin'].isin(df_googleAPI['origin'])]
    
    # evaluate MSEs and return value
    proximity_ratio_mse = mean_squared_error(df_googleAPI['proximity_ratio_test'], df_psrc_reduced['proximity_ratio_test'])
    vert_hori_ratio_mse = mean_squared_error(df_googleAPI['vertical_horizontal_ratio_test'], df_psrc_reduced['vertical_horizontal_ratio_test'])
    average_distance_mse = mean_squared_error(df_googleAPI['average_distance_test'], df_psrc_reduced['average_distance_test'])

    print("proximity ratio mse: ", proximity_ratio_mse)
    print("vertical horizontal ratio mse: ", vert_hori_ratio_mse)
    print("average distance mse: ", average_distance_mse)
    result = [proximity_ratio_mse, vert_hori_ratio_mse, average_distance_mse]
    # Don't forget to normalize the values in order to compare them!
     
    return (result)   

In [9]:
# Load PSRC data and pre-process; column names should be determined at a group meeting
psrc_rawdat = pd.read_csv("psrc_data_final.csv", dtype={'o_bg': str, 'd_bg': str})
psrc_rawdat['origin'] = '530330' + psrc_rawdat['o_bg']
psrc_rawdat['destination'] = '530330' + psrc_rawdat['d_bg']
psrc_orig_pts = psrc_rawdat['o_bg_lat_long'].apply(shapely.wkt.loads)
psrc_dest_pts = psrc_rawdat['d_bg_lat_long'].apply(shapely.wkt.loads)
psrc_rawdat['orig_lng'] = pd.DataFrame([kk.x for kk in psrc_orig_pts])
psrc_rawdat['orig_lat'] = pd.DataFrame([kk.y for kk in psrc_orig_pts])
psrc_rawdat['dest_lng'] = pd.DataFrame([kk.x for kk in psrc_dest_pts])
psrc_rawdat['dest_lat'] = pd.DataFrame([kk.y for kk in psrc_dest_pts])
psrc_rawdat.drop(columns = ['o_bg', 'o_bg_lat_long', 'd_bg', 'd_bg_lat_long'], inplace=True)
psrc_rawdat.rename(columns = {'trip_path_distance': 'distance', 'trip_weight_revised': 'weight'}, inplace=True)
psrc_rawdat['distance'] = pd.to_numeric(psrc_rawdat['distance'], errors='coerce')

In [10]:
# Load Google API data 
# df_sample = pd.read_csv('Proximity_Ratio.csv', dtype={'bg_origin': str}) 
input_destinations = pd.read_csv('GoogleMatrix_Places_Dist.csv', dtype={'origin': str})
input_destinations.rename(columns = {'lat': 'dest_lat', 'lng': 'dest_lng'}, inplace=True)

In [11]:
# Load blockgroup data with latitude and longitudes; will be merged with Google API
blockgroup_mapping = pd.read_csv('SeattleCensusBlockGroups.csv', dtype={'tract_blkgrp': str})
blockgroup_mapping['tract_blkgrp'] = '530330' + blockgroup_mapping['tract_blkgrp']
orig_pts = blockgroup_mapping.centroid.apply(shapely.wkt.loads)
blockgroup_mapping['orig_lng'] = pd.DataFrame([kk.x for kk in orig_pts])
blockgroup_mapping['orig_lat'] = pd.DataFrame([kk.y for kk in orig_pts])
origin_blockgroups = blockgroup_mapping [['tract_blkgrp', 'orig_lat', 'orig_lng']]

In [12]:
# origin_merged will be an input data for 'evaluate_features' function
origin_merged = pd.merge(left=input_destinations, right=origin_blockgroups, how='left', left_on='origin', right_on='tract_blkgrp')
origin_merged = origin_merged[['origin', 'dest_lat', 'orig_lat','dest_lng', 'orig_lng', 'rank', 'distance', 'class']]

In [13]:
"""
You can play around here!
"""
testArray = [3, 5, 2, 2, 2, 2, 3, 0, 1, 2]
#testArray = [0, 8, 0, 0, 0, 0, 0, 0, 0, 3]
df_psrc = prepare_psrc(psrc_rawdat)
evaluate_features(origin_merged, df_psrc, testArray)

proximity ratio mse:  0.823309313184
vertical horizontal ratio mse:  248.891308226
average distance mse:  0.987363131716


[0.82330931318383938, 248.89130822615772, 0.9873631317158188]